In [1]:
!pip install wandb --quiet
!pip install scikit-image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import gc

import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
import numpy

import torch
from torchsummary import summary
import torchvision
import skimage
from skimage import color
from skimage import transform

from torch.utils.data import DataLoader

from PIL import Image
from tqdm import tqdm
from torch.utils.data import Dataset

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import glob
import wandb
import matplotlib.pyplot as plt

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", DEVICE)

Device:  cuda


In [3]:
from google.colab import drive # Link your drive if you are a colab user
drive.mount('/content/drive') # Models in this HW take a long time to get trained and make sure to save it her

Mounted at /content/drive


In [ ]:
f_write = open('/content/drive/MyDrive/dataset/spilt_list/valid_neg.txt', 'w')
with open('/content/drive/MyDrive/dataset/spilt_list/negative_valid.txt', 'r') as f:
  lines = f.readlines()
  for line in lines:
    line = line.rstrip()
    f_write.write(line)
    f_write.write(" ")
    f_write.write("0")
    f_write.write("\n")

f_write.close()


In [ ]:
l= []
folders = os.listdir(root)
for i in folders:
  k = os.listdir(root+i)
  l.append(len(k)) 

In [17]:
gc.collect()
torch.cuda.empty_cache()

In [9]:
class MyDataset(Dataset):
  def __init__(self, txt_path, root,transform=None, target_transform=None): 
      fh = open(txt_path, 'r') 
      imgs = []  
      for line in fh:
          line = line.rstrip()
          words = line.split() 
          imgs.append((words[0], int(words[1]))) 

      self.imgs = imgs  
      self.transform = transform
      self.target_transform = target_transform
      self.root = root

  def __getitem__(self, index):
      fd, label = self.imgs[index]
      fd_path = self.root + fd
      files = os.listdir(fd_path)
      imgs = []
      for fn in files:
        img = Image.open(fd_path+ '/'+ fn).convert('L')
        if self.transform is not None:
          img = self.transform(img)
        imgs.append(img)
      imgs_batch =  torch.stack(imgs,dim=1)
      imgs_batch = transform.resize(imgs_batch,(64,256,256))
      label = torch.Tensor(label)
      return imgs_batch, label

  def __len__(self):
      return len(self.imgs)   

In [19]:
train_txtpath = '/content/drive/MyDrive/dataset/spilt_list/train.txt'
valid_txtpath = '/content/drive/MyDrive/dataset/spilt_list/valid.txt'
root = '/content/drive/MyDrive/dataset/png_data/'

train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
])

valid_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
])

valid_dataset =  MyDataset(valid_txtpath,root,valid_transforms)
train_dataset = MyDataset(train_txtpath,root,train_transforms)

In [20]:
config = {
    'batch_size': 2, # Increase this if your GPU can handle it
    'lr': 0.01,
    'epochs': 20, # 10 epochs is recommended ONLY for the early submission - you will have to train for much longer typically.
    # Include other parameters as needed.
}

In [21]:
train_loader = torch.utils.data.DataLoader(
    dataset     = train_dataset, 
    batch_size  = config['batch_size'], 
    shuffle     = True,
    num_workers = 2, 
    pin_memory  = True
)

valid_loader = torch.utils.data.DataLoader(
    dataset     = valid_dataset, 
    batch_size  = config['batch_size'],
    shuffle     = False,
    num_workers = 2
)

In [22]:
for i, (images, labels) in enumerate(train_loader):
  print(images.shape)
  print(labels.shape)
  break

RuntimeError: ignored

In [2]:
import torch.nn as nn
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        self.conv_layer1 = self._conv_layer_set(1, 32)
        self.conv_layer2 = self._conv_layer_set(32, 64)
        self.fc1 = nn.Linear(2**3*64, 128)
        self.fc2 = nn.Linear(128, 2)
        self.relu = nn.LeakyReLU()
        self.batch=nn.BatchNorm1d(128)
        self.drop=nn.Dropout(p=0.15)        
        
    def _conv_layer_set(self, in_c, out_c):
        conv_layer = nn.Sequential(
        nn.Conv3d(in_c, out_c, kernel_size=(3, 3, 3), padding=0),
        nn.LeakyReLU(),
        nn.MaxPool3d((2, 2, 2)),
        )
        return conv_layer
    

    def forward(self, x):
        # Set 1
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.batch(out)
        out = self.drop(out)
        out = self.fc2(out)
        print(out.shape)
        
        return out

In [3]:
model = CNNModel()
inp = torch.randn(1,1,64,256,256)
model(inp)


NameError: ignored